In [1]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/bootiyahou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
data_path = "../data/txt/"

# Choix d'une décennie et du nombre de clusters

In [4]:
DECADE = '1860'
N_CLUSTERS = 10

# Chargement des fichiers de la décennie

In [5]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [6]:
# Exemple de fichiers
files

['Bxl_1860_Tome_I1_Part_1.txt',
 'Bxl_1860_Tome_I1_Part_2.txt',
 'Bxl_1860_Tome_I1_Part_3.txt',
 'Bxl_1860_Tome_I1_Part_4.txt',
 'Bxl_1860_Tome_II1_Part_1.txt',
 'Bxl_1860_Tome_II1_Part_2.txt',
 'Bxl_1860_Tome_II1_Part_3.txt',
 'Bxl_1860_Tome_II1_Part_4.txt',
 'Bxl_1860_Tome_II1_Part_5.txt',
 'Bxl_1860_Tome_II1_Part_6.txt',
 'Bxl_1861_Tome_II1_Part_1.txt',
 'Bxl_1861_Tome_II1_Part_2.txt',
 'Bxl_1861_Tome_II1_Part_3.txt',
 'Bxl_1861_Tome_II1_Part_4.txt',
 'Bxl_1861_Tome_II1_Part_5.txt',
 'Bxl_1862_Tome_I1_Part_1.txt',
 'Bxl_1862_Tome_I1_Part_2.txt',
 'Bxl_1862_Tome_I1_Part_3.txt',
 'Bxl_1862_Tome_I1_Part_4.txt',
 'Bxl_1862_Tome_I1_Part_5.txt',
 'Bxl_1862_Tome_II1_Part_1.txt',
 'Bxl_1862_Tome_II1_Part_2.txt',
 'Bxl_1862_Tome_II1_Part_3.txt',
 'Bxl_1862_Tome_II1_Part_4.txt',
 'Bxl_1862_Tome_II1_Part_5.txt',
 'Bxl_1863_Tome_I1_Part_1.txt',
 'Bxl_1863_Tome_I1_Part_2.txt',
 'Bxl_1863_Tome_I1_Part_3.txt',
 'Bxl_1863_Tome_II1_Part_1.txt',
 'Bxl_1863_Tome_II1_Part_2.txt',
 'Bxl_1863_Tome_II1_Pa

In [7]:
texts = [open(data_path + f).read() for f in files]

In [8]:
# Exemple de textes
texts[0][:400]

"ULLETIN COMMUNAL.\nAnnée\n\nPREMIER\n\n1860.\n\nSEMESTRE.\n\nB R U X E L L E S ,\nIMPRIMERIE\n\nDE\n\nBOLS-W1TTOUCK,\n\nRUE\n\nAU\n\nLAIT\n\n\x0c\x0cVILLE\n\nDE BRUXELLES\n\nBULLETIN COMMUNAL.\nANNÉE\n\n1860.\n\nNUMÉRO 1.\n\nMARDI 2 i JANVIER.\n\nOuverture de rues nouvelles au quartier Lcopold.\nEnquête.\nPar délibération du 51 décembre 1859, le Conseil communal a\nadopté un plan pour l'ouverture de rues nouvelles au quartier\nLéopold.\nCe pl"

# Vectorisation du texte

In [9]:
def process_text(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

In [10]:
vectorizer = TfidfVectorizer(tokenizer=process_text,
                                stop_words=stopwords.words('french'),
                                max_df=0.5,
                                min_df=0.1,
                                lowercase=True)

In [11]:
%time tfidf_vectors = vectorizer.fit_transform(texts)

CPU times: user 25.2 s, sys: 91.8 ms, total: 25.3 s
Wall time: 25.4 s


In [12]:
tfidf_vectors

<94x8619 sparse matrix of type '<class 'numpy.float64'>'
	with 175985 stored elements in Compressed Sparse Row format>

In [13]:
# Exemple de vecteur TFIDF
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names()
    ).sort_values(ascending=False)

beurre              0.572521
n.-d.-aux-neiges    0.284931
compteurs           0.184364
vanderlinden        0.180411
épingles            0.172886
                      ...   
manifestée          0.000000
manière+suivante    0.000000
manquants           0.000000
manque              0.000000
#                   0.000000
Length: 8619, dtype: float64

# Comprendre les vecteurs et leurs "distances"

In [14]:
from scipy.spatial.distance import cosine

In [15]:
cosine([1, 2, 3], [1, 2, 3])

0.0

In [16]:
cosine([1, 2, 3], [1, 2, 2])

0.02004211298777725

In [17]:
cosine([1, 2, 3], [2, 2, 2])

0.07417990022744858

In [18]:
tfidf_array = tfidf_vectors.toarray()

In [19]:
tfidf_array[0]

array([0.        , 0.        , 0.        , ..., 0.01579924, 0.        ,
       0.00768182])

In [20]:
texts[0][:200]

'ULLETIN COMMUNAL.\nAnnée\n\nPREMIER\n\n1860.\n\nSEMESTRE.\n\nB R U X E L L E S ,\nIMPRIMERIE\n\nDE\n\nBOLS-W1TTOUCK,\n\nRUE\n\nAU\n\nLAIT\n\n\x0c\x0cVILLE\n\nDE BRUXELLES\n\nBULLETIN COMMUNAL.\nANNÉE\n\n1860.\n\nNUMÉRO 1.\n\nMARDI 2 i JANV'

In [21]:
cosine(tfidf_array[0], tfidf_array[1])

0.4992974485061431

# Clustering des vecteurs TFIDF

Article intéressant sur le KMeans clustering :
- https://medium.com/dataseries/k-means-clustering-explained-visually-in-5-minutes-b900cc69d175

In [22]:
km_model = KMeans(n_clusters=N_CLUSTERS)

In [23]:
km_model.fit(tfidf_vectors)

KMeans(n_clusters=10)

In [24]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(km_model.labels_):
    clustering[label].append(files[idx])

In [25]:
pprint(dict(clustering))

{0: ['Bxl_1861_Tome_II1_Part_3.txt',
     'Bxl_1865_Tome_II1_Part_3.txt',
     'Bxl_1866_Tome_II1_Part_3.txt',
     'Bxl_1866_Tome_II1_Part_4.txt',
     'Bxl_1867_Tome_II1_Part_1.txt',
     'Bxl_1867_Tome_II1_Part_2.txt',
     'Bxl_1868_Tome_II1_Part_2.txt',
     'Bxl_1868_Tome_II1_Part_5.txt',
     'Bxl_1869_Tome_II1_Part_1.txt',
     'Bxl_1869_Tome_II1_Part_2.txt'],
 1: ['Bxl_1865_Tome_II1_Part_5.txt',
     'Bxl_1866_Tome_I1_Part_1.txt',
     'Bxl_1867_Tome_I1_Part_1.txt',
     'Bxl_1867_Tome_I1_Part_4.txt',
     'Bxl_1867_Tome_II1_Part_4.txt',
     'Bxl_1867_Tome_II1_Part_5.txt',
     'Bxl_1868_Tome_I1_Part_2.txt',
     'Bxl_1868_Tome_I1_Part_3.txt',
     'Bxl_1868_Tome_II1_Part_1.txt',
     'Bxl_1868_Tome_II1_Part_3.txt',
     'Bxl_1868_Tome_II1_Part_4.txt'],
 2: ['Bxl_1861_Tome_II1_Part_1.txt',
     'Bxl_1861_Tome_II1_Part_4.txt',
     'Bxl_1861_Tome_II1_Part_5.txt',
     'Bxl_1862_Tome_I1_Part_1.txt',
     'Bxl_1862_Tome_I1_Part_2.txt',
     'Bxl_1862_Tome_I1_Part_3.txt',
     'B